[2024-12-13 Fiddler](https://thefiddler.substack.com/p/can-you-survive-the-floor)
====================

Fiddler
-------
I think it's clear that a corner has the best chance of avoiding eliminations, maximizing the
probability of winning.

In the first round, a corner has $1/9 + 2/9 \cdot 1/3$ chance of facing elimination, while
a side has $1/9 + 2/9 \cdot 1/2 + 1/9 \cdot 1/4$ and the middle has $1/9 + 4/9 \cdot 1/3$.

Calculating the win probabilities confirms that the corner has the best chance of winning.

Extra credit
------------
The only relevant parameter for each round is the contestant graph.  The exact topology is
irrelevant after the initial contestant graph is determined.  After an elimination, the loser's
neighbors become the winner's neighbors.

In [1]:
from functools import cache

def initial_graph(n):
    def initial_neighbors(i,j):
        neighbors = []
        if i > 1:
            neighbors.append((i-1,j))
        if j > 1:
            neighbors.append((i,j-1))
        if j < n:
            neighbors.append((i,j+1))
        if i < n:
            neighbors.append((i+1,j))
        return frozenset(neighbors)
    return tuple([((i,j),initial_neighbors(i,j)) for i in [1..n] for j in [1..n]])

def matchup_probability(a,b,graph):
    if a[0] not in b[1]:
        return 0
    return (1/len(a[1]) + 1/len(b[1]))/len(graph)

def new_graph(winner,loser,graph):
    g = []
    for c in graph:
        if c == winner:
            g.append((c[0], frozenset((c[1]-{loser[0]}) | (loser[1]-{c[0]}))))
        elif c == loser:
            pass
        elif loser[0] in c[1]:
            g.append((c[0], frozenset((c[1]-{loser[0]}) | {winner[0]})))
        else:
            g.append(c)
    return tuple(g)
            
@cache
def win_probability(contestant, graph):
    if len(graph) == 2:
        return 1/2
    p = 0
    for a in graph:
        for b in graph:
            if a[0] >= b[0]:
                continue
            if a[0] == contestant:
                p += 1/2*matchup_probability(a,b,graph)*win_probability(contestant,new_graph(a,b,graph))
            elif b[0] == contestant:
                p += 1/2*matchup_probability(a,b,graph)*win_probability(contestant,new_graph(b,a,graph))
            else:
                # it doesn't matter who won, choose a
                p += matchup_probability(a,b,graph)*win_probability(contestant,new_graph(a,b,graph))
    return p

In [2]:
probs = [(c,win_probability(c,initial_graph(3))) for c in [(1,1),(2,1),(2,2)]]
[(c,numerical_approx(p),p) for (c,p) in probs]

[((1, 1), 0.128909366933294, 95802636437/743178240000),
 ((2, 1), 0.0994265264956878, 8977833162901/90296156160000),
 ((2, 2), 0.0866564262840710, 3912371100007/45148078080000)]

Corner: 95802636437/743178240000 ≈ 12.9%

Side: 8977833162901/90296156160000 ≈ 9.9%

Center: 3912371100007/45148078080000 ≈ 8.7%

In [3]:
4*probs[0][1]+4*probs[1][1]+probs[2][1]

1

Running [simulations](20241213.go) agree:

    $ go run 20241213.go
    1,1: 0.129202
    1,2: 0.099212
    1,3: 0.129183
    2,1: 0.100070
    2,2: 0.086032
    2,3: 0.099874
    3,1: 0.128740
    3,2: 0.098840
    3,3: 0.128846